# 思维链演进实验：CoT → ToT → GoT

基于吴恩达 x OpenAI 提示词工程课程的实践 notebook

## 📚 什么是思维链？

思维链（Chain of Thought）是一类让模型**展示推理过程**而不是直接给出答案的提示技术。

这些技术模拟了人类解决复杂问题时的思考方式：
- 将大问题分解为小步骤
- 探索多种可能的路径
- 综合不同角度的信息

## 🔄 演进过程

```
CoT (2022)          ToT (2023)          GoT (2023)
线性思维链    →    树状思维探索    →    图状思维网络
  单路径            多路径+回溯         多路径+聚合
```

### 1️⃣ CoT (Chain of Thought) - 思维链

**定义**：让模型逐步展示推理过程

**核心思想**：将复杂问题分解为一系列中间步骤

```
[问题] → [步骤1] → [步骤2] → [步骤3] → [答案]
```

**两种实现方式**：
- **Zero-shot CoT**: "Let's think step by step."（让我们一步步思考）
- **Few-shot CoT**: 提供带推理过程的示例

### 2️⃣ ToT (Tree of Thoughts) - 思维树

**定义**：将推理组织为树状结构，探索多个路径并评估

**核心思想**：像下棋一样进行"前瞻性搜索"，可以回溯和剪枝

```
                [问题]
                  |
      +-----------+-----------+
      |           |           |
   [思路1]     [思路2]     [思路3]
    得分:7      得分:9      得分:4
      |           |           ×(剪枝)
   +--+--+     +--+--+
   |     |     |     |
 [子1] [子2] [子3] [子4]
  ×     ×    ✓(最优)  ×
```

### 3️⃣ GoT (Graph of Thoughts) - 思维图

**定义**：将思维组织为图状结构，允许思路之间相互连接、合并和聚合

**核心思想**：思维单元可以任意组合、拆分、聚合

```
[子问题1] ──┐
             ├──> [聚合思路] ──> [最终答案]
[子问题2] ──┘          ↑
    ↓                  |
[中间结果] ─────────────┘
```

**三种操作**：
- **Aggregate（聚合）**：合并多个思路
- **Refine（精炼）**：改进现有思路
- **Generate（生成）**：创建新的分支

## 🎯 三者对比

| 特性 | CoT | ToT | GoT |
|-----|-----|-----|-----|
| **结构** | 线性链条 | 树状结构 | 图状网络 |
| **路径** | 单一路径 | 多路径+剪枝 | 多路径+聚合 |
| **回溯** | ❌ 不支持 | ✅ 支持 | ✅ 支持 |
| **聚合** | ❌ 不支持 | ❌ 不支持 | ✅ 支持 |
| **实现复杂度** | 低 | 中 | 高 |
| **适用场景** | 线性推理 | 搜索问题 | 复杂综合 |

## 🤖 测试模型

使用 **Qwen3 8B** 模型

## 💡 与推理模型的区别

⚠️ **重要**：CoT/ToT/GoT 是**提示工程技巧**，而推理模型（o1、DeepSeek-R1、Claude Extended Thinking）的思维链是**内置能力**。

| 维度 | 提示词思维链 (CoT/ToT/GoT) | 推理模型 (o1/R1) |
|------|---------------------------|------------------|
| **本质** | 提示工程技巧 | 模型内置能力 |
| **谁在思考** | 用户引导模型 | 模型自主思考 |
| **自我验证** | ❌ 无 | ✅ 有 |
| **回溯能力** | ❌ 需多轮模拟 | ✅ 内置 |
| **实现方式** | 优化 prompt | 训练时植入 |
| **出现时间** | 2022-2023 | 2024-2025 |

---

## 第一步：环境准备

安装并启动 Ollama 服务

In [10]:
# === 1. 在 Colab 安装并启动 Ollama ===
import subprocess
import time

print("⬇️ 正在安装 Ollama...")
subprocess.run("curl -fsSL https://ollama.com/install.sh | sh", shell=True, check=False)
print("✅ Ollama 安装完成")

print("🚀 正在后台启动 Ollama 服务...")
ollama_server = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
print("✅ Ollama 服务已在 localhost:11434 运行")

⬇️ 正在安装 Ollama...
✅ Ollama 安装完成
🚀 正在后台启动 Ollama 服务...
✅ Ollama 服务已在 localhost:11434 运行


In [2]:
# === 2. 下载模型并导入核心函数 ===
import subprocess
import requests
import json
import time

print("⬇️ 正在下载 qwen3:8b...")
subprocess.run(["ollama", "pull", "qwen3:8b"], check=False)
print("✅ 模型下载完成！")

# 定义核心函数
OLLAMA_URL = "http://localhost:11434"

def get_completion(prompt, model="qwen3:8b", temperature=0.7, show_output=True, enable_thinking=False):
    """
    调用模型获取回复

    参数:
        enable_thinking: 是否开启 Qwen3 内置思考模式（默认 False 关闭）

    注意:
        - Qwen3 默认会思考，如果不显式传 think=False，模型会进入思考模式
        - 思考模式下，thinking 字段有内容但 content 可能为空
        - 本函数会检测 thinking 字段，避免因未接收思考内容导致超时
    """
    url = f"{OLLAMA_URL}/api/chat"
    options = {"temperature": temperature, "top_p": 0.9, "num_ctx": 8192, "num_predict": 4096}
    messages = [{"role": "user", "content": prompt}]
    # ⚠️ 关键：显式传 think 参数控制思考模式
    payload = {
        "model": model,
        "messages": messages,
        "stream": True,
        "options": options,
        "think": enable_thinking  # 默认关闭思考，避免超时
    }

    try:
        response = requests.post(url, json=payload, stream=True)
        response.raise_for_status()
        result = ""
        thinking_text = ""
        is_thinking_shown = False

        if show_output:
            print("🤖 模型回复: ", end='', flush=True)

        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line.decode('utf-8'))

                    if 'message' in data:
                        # 检测并处理 thinking 字段（即使 enable_thinking=False 也要检测）
                        if 'thinking' in data['message'] and data['message']['thinking']:
                            thinking_content = data['message']['thinking']
                            thinking_text += thinking_content
                            if show_output and not is_thinking_shown:
                                print(f"\n💭 [检测到思考过程，长度: {len(thinking_text)}+]", end='', flush=True)
                                is_thinking_shown = True

                        # 处理 content 字段
                        if 'content' in data['message'] and data['message']['content']:
                            content = data['message']['content']
                            if show_output:
                                if is_thinking_shown:
                                    print(f"\n✅ [正式回答]: ", end='', flush=True)
                                    is_thinking_shown = False
                                print(content, end='', flush=True)
                            result += content

                    if data.get('done', False):
                        if show_output:
                            print()
                        break

                except json.JSONDecodeError:
                    continue

        # 如果 content 为空但 thinking 有内容，说明模型在思考但没有正式输出
        if not result.strip() and thinking_text.strip():
            if show_output:
                print(f"⚠️ content 为空，检测到 thinking 内容（{len(thinking_text)} 字符）")
                print(f"💡 这说明模型默认开启了思考模式，但 think 参数未正确传递")
            # 可以选择使用 thinking 内容作为结果
            result = f"[思考内容]: {thinking_text[:500]}..." if len(thinking_text) > 500 else f"[思考内容]: {thinking_text}"

        return result.strip()
    except Exception as e:
        print(f"\n❌ 错误: {str(e)}")
        return f"[错误: {str(e)}]"

print("✅ 核心函数加载完成（支持 think 参数控制）")
print("💡 默认 enable_thinking=False 关闭思考模式，避免超时")

⬇️ 正在下载 qwen3:8b...
✅ 模型下载完成！
✅ 核心函数加载完成（支持 think 参数控制）
💡 默认 enable_thinking=False 关闭思考模式，避免超时


---

# 实验一：CoT（思维链）- 线性推理

## 📖 任务说明

**任务**：解决复杂数学应用题

**测试问题**：
> 一家书店举行促销活动。原价120元的书打8折，满100元再减20元，最后使用会员卡再打9折。小明买了这本书，实际支付了多少钱？

## 🎯 三种方法对比

1. **直接回答**：不引导推理过程
2. **Zero-shot CoT**：使用魔法咒语 \"Let's think step by step\"
3. **Few-shot CoT**：提供带推理过程的示例

## 💡 预期效果

- 直接回答可能出错或跳步
- Zero-shot CoT 会展示推理过程，准确率提升
- Few-shot CoT 推理格式更规范

In [3]:
# 实验一：CoT（思维链）对比

# 测试问题
math_problem = """
一家书店举行促销活动。原价120元的书打8折，满100元再减20元，
最后使用会员卡再打9折。小明买了这本书，实际支付了多少钱？
"""

print("="*80)
print("🔬 实验一：CoT（思维链）- 线性推理")
print("="*80)
print(f"测试问题：{math_problem.strip()}")
print("="*80)

# 方法1：直接回答（无引导）
print("\n❌ 方法1：直接回答（无引导）")
print("─"*80)
direct_prompt = f"""
请回答以下问题：

{math_problem}

答案："""
result_direct = get_completion(direct_prompt, temperature=0.7)
print(f"\n📊 输出长度: {len(result_direct)} 字符")

time.sleep(2)

# 方法2：Zero-shot CoT（魔法咒语）
print("\n✅ 方法2：Zero-shot CoT（让我们一步步思考）")
print("─"*80)
zero_shot_cot_prompt = f"""
请回答以下问题：

{math_problem}

让我们一步步思考。(Let's think step by step.)
"""
result_zero_shot = get_completion(zero_shot_cot_prompt, temperature=0.7)
print(f"\n📊 输出长度: {len(result_zero_shot)} 字符")

time.sleep(2)

# 方法3：Few-shot CoT（提供示例）
print("\n✅ 方法3：Few-shot CoT（提供推理示例）")
print("─"*80)
few_shot_cot_prompt = f"""
请按照以下示例的格式，逐步推理并回答问题。

[示例]
问题：一件衣服原价200元，先打7折，再满150元减30元，最终价格是多少？
推理过程：
步骤1：计算打折后的价格
  200 × 0.7 = 140元
步骤2：检查是否满足满减条件
  140元 < 150元，不满足满减条件
步骤3：最终价格
  最终价格 = 140元

现在请回答：
{math_problem}

推理过程：
"""
result_few_shot = get_completion(few_shot_cot_prompt, temperature=0.7)
print(f"\n📊 输出长度: {len(result_few_shot)} 字符")

print("\n" + "="*80)
print("💡 观察要点：")
print("  - 直接回答：可能跳过步骤或出错")
print("  - Zero-shot CoT：会展示推理过程")
print("  - Few-shot CoT：推理格式更规范，步骤更清晰")
print("="*80 + "\n")

🔬 实验一：CoT（思维链）- 线性推理
测试问题：一家书店举行促销活动。原价120元的书打8折，满100元再减20元，
最后使用会员卡再打9折。小明买了这本书，实际支付了多少钱？

❌ 方法1：直接回答（无引导）
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 我们来一步一步计算小明实际支付的金额：

1. **原价**：120元  
2. **打8折**：  
   $ 120 \times 0.8 = 96 $ 元  

3. **满100元再减20元**：  
   因为 96 元 < 100 元，所以**不满足减20元的条件**，不减。

4. **使用会员卡再打9折**：  
   $ 96 \times 0.9 = 86.4 $ 元  

---

**最终答案：86.4元**

📊 输出长度: 218 字符

✅ 方法2：Zero-shot CoT（让我们一步步思考）
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 我们来一步步计算小明买这本书的实际支付金额。

---

### **第一步：原价打8折**

原价是：**120元**

打8折，即：  
$$
120 \times 0.8 = 96 \text{元}
$$

---

### **第二步：满100元再减20元**

现在价格是：**96元**

因为 **96 < 100**，所以**不满足“满100元再减20元”的条件**，所以**不减**。

---

### **第三步：使用会员卡再打9折**

当前价格是：**96元**

打9折，即：  
$$
96 \times 0.9 = 86.4 \text{元}
$$

---

### **最终答案：**

小明实际支付了：  
$$
\boxed{86.4} \text{元}
$$

📊 输出长度: 350 字符

✅ 方法3：Few-shot CoT（提供推理示例）
─────────────────────────────────────────────────────

In [7]:
# 额外实验：Qwen3 内置思考模式 vs 提示词 CoT

# 使用 HTTP 接口直连 Ollama，避免依赖 python 包
import requests
import json
import time

# 复用或兜底设置 Ollama 服务地址
try:
    OLLAMA_URL
except NameError:
    OLLAMA_URL = "http://localhost:11434"


def get_completion_with_think(prompt, model="qwen3:8b", think=False, show_output=True):
    """调用模型（支持 think 参数，HTTP 直连 Ollama）"""
    url = f"{OLLAMA_URL}/api/chat"
    messages = [{"role": "user", "content": prompt}]

    # 构造参数 - think 参数控制内置思考模式
    payload = {
        "model": model,
        "messages": messages,
        "stream": True,
        "think": bool(think),
    }

    try:
        resp = requests.post(url, json=payload, stream=True, timeout=300)
        resp.raise_for_status()

        result = ""
        thinking_part = ""
        answer_part = ""
        is_thinking = False

        if show_output:
            print("🤖 模型回复: ", end='', flush=True)

        for line in resp.iter_lines():
            if not line:
                continue
            try:
                data = json.loads(line.decode("utf-8"))
            except json.JSONDecodeError:
                continue

            msg = data.get("message") or {}

            # 处理思考部分（thinking 字段）
            thinking_content = msg.get("thinking") or ""
            if thinking_content:
                if show_output:
                    if not is_thinking:
                        print("\n\n💭 [思考过程]", flush=True)
                        print("─" * 80, flush=True)
                        is_thinking = True
                    print(thinking_content, end='', flush=True)
                thinking_part += thinking_content

            # 处理回答部分（content 字段）
            content = msg.get("content") or ""
            if content:
                if is_thinking and show_output:
                    print("\n" + "─" * 80, flush=True)
                    print("✅ [正式回答]\n", flush=True)
                    is_thinking = False
                if show_output:
                    print(content, end='', flush=True)
                result += content
                answer_part += content

            if data.get("done"):
                break

        if show_output:
            print()

        # 若 content 为空但存在 thinking，给出提示（不强行用思考内容充当答案）
        if not result.strip() and thinking_part.strip() and show_output:
            print(f"⚠️ content 为空，但检测到 thinking（{len(thinking_part)} 字符）")

        return result.strip()

    except requests.exceptions.ConnectionError:
        msg = "无法连接到 Ollama 服务，请确认本地已启动：`ollama serve`，并已拉取模型：`ollama pull qwen3:8b`"
        if show_output:
            print(f"\n❌ 错误: {msg}")
        return f"[错误: {msg}]"
    except Exception as e:
        if show_output:
            print(f"\n❌ 错误: {str(e)}")
        return f"[错误: {str(e)}]"



# 测试问题（复杂数学题）
test_question = """
一个水池有甲、乙两个进水管。单独开甲管，12小时可以注满水池；
单独开乙管，18小时可以注满水池。如果两管同时开放，多少小时可以注满水池？
"""

print("="*80)
print("🔬 额外实验：Qwen3 内置思考模式 vs 提示词 CoT")
print("="*80)
print(f"测试问题：{test_question.strip()}")
print("="*80)

# 方法1：Qwen3 关闭思考（普通模式）
print("\n📌 方法1：Qwen3 普通模式（think=False，直接回答）")
print("─"*80)
result_no_think = get_completion_with_think(
    prompt=f"请回答：{test_question}\n答案：",
    think=False
)
print(f"\n📊 输出长度: {len(result_no_think)} 字符")

time.sleep(2)

# 方法2：Qwen3 关闭思考 + CoT 提示词
print("\n📌 方法2：提示词 CoT（think=False + CoT 提示词）")
print("─"*80)
result_no_think_cot = get_completion_with_think(
    prompt=f"""请回答：{test_question}

让我们一步步思考。(Let's think step by step.)
""",
    think=False
)
print(f"\n📊 输出长度: {len(result_no_think_cot)} 字符")

time.sleep(2)

# 方法3：Qwen3 内置思考模式（think=True）- 使用简单问题避免超时
print("\n📌 方法3：Qwen3 内置思考（think=True，简单测试）")
print("─"*80)
print(f"⚠️ 使用问题测试：{test_question}")
print()
result_with_think = get_completion_with_think(
    prompt=f"请回答：{test_question}",
    think=True  # 开启内置思考模式
)
print(f"\n📊 输出长度: {len(result_with_think)} 字符")
print()
print("💡 说明：think=True 会让模型：")
print("   1. 在 'thinking' 字段中展示思考过程")
print("   2. 在 'content' 字段中给出最终答案")
print("   3. 思考过程对用户可见（通过 API 返回）")
print("─"*80)

time.sleep(2)



🔬 额外实验：Qwen3 内置思考模式 vs 提示词 CoT
测试问题：一个水池有甲、乙两个进水管。单独开甲管，12小时可以注满水池；
单独开乙管，18小时可以注满水池。如果两管同时开放，多少小时可以注满水池？

📌 方法1：Qwen3 普通模式（think=False，直接回答）
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 我们来分析这个问题：

- 甲管单独开，12小时注满水池，说明甲管每小时注水的量是：  
  $ \frac{1}{12} $ 水池。

- 乙管单独开，18小时注满水池，说明乙管每小时注水的量是：  
  $ \frac{1}{18} $ 水池。

两管同时开放时，每小时注水的总量是：

$$
\frac{1}{12} + \frac{1}{18}
$$

我们来通分计算：

$$
\frac{1}{12} + \frac{1}{18} = \frac{3}{36} + \frac{2}{36} = \frac{5}{36}
$$

所以，两管同时开放时，每小时注水 $ \frac{5}{36} $ 水池。

要注满一整个水池，所需时间为：

$$
\frac{1}{\frac{5}{36}} = \frac{36}{5} = 7.2 \text{ 小时}
$$

**答案：7.2小时**

📊 输出长度: 401 字符

📌 方法2：提示词 CoT（think=False + CoT 提示词）
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 我们来一步一步地解决这个问题。

---

### **第一步：理解题意**

- **甲管单独开**，12小时可以注满水池；
- **乙管单独开**，18小时可以注满水池；
- 问：**甲乙两管同时开放**，需要多少小时才能注满水池？

---

### **第二步：设总水量为单位1**

我们可以把整个水池的水量看作是 **1** 个单位。

---

### **第三步：计算甲、乙两管的注水速度**

- **甲管的注水速度**：每小时注

## 🔬 额外实验：Qwen3 内置思考模式 vs 提示词 CoT

Qwen3 是一个可以开启/关闭思考模式的推理模型。我们可以对比三种情况：

1. **Qwen3 关闭思考**（`think=False`）：作为普通模型使用
2. **Qwen3 关闭思考 + CoT 提示词**：用提示词引导思考
3. **Qwen3 开启内置思考**（`think=True`）：使用模型自带的推理能力

这个对比能清晰展示**提示词工程 vs 模型内置能力**的本质区别！

---

# 实验二：ToT（思维树）- 多路径探索

## 📖 任务说明

**任务**：24点游戏 - 用给定的4个数字通过加减乘除得到24

**测试问题**：
> 用 4, 9, 10, 13 这四个数字，通过加减乘除运算得到24

## 🎯 ToT 工作流程

```
                [初始问题]
                    |
    +---------------+---------------+
    |               |               |
[思路1]         [思路2]         [思路3]
 评估得分         评估得分         评估得分
    |               |               |
选择最优 ────────> [验证] ────────> [答案]
```

## 💡 核心机制

1. **生成多个思路**：探索不同的组合方式
2. **评估每个思路**：检查是否能得到24，打分
3. **选择最优路径**：选择得分最高的思路
4. **剪枝无效路径**：放弃得分低的思路

## 🔍 对比 CoT 和 ToT

- **CoT**：线性思考，一条路走到底
- **ToT**：树状探索，可以回溯和选择

In [5]:
# 实验二：ToT（思维树）多路径探索

# 测试问题：24点游戏
numbers_24 = "4, 9, 10, 13"

print("="*80)
print("🔬 实验二：ToT（思维树）- 多路径探索")
print("="*80)
print(f"测试问题：用 {numbers_24} 这四个数字通过加减乘除得到 24")
print("="*80)

# 对比1：CoT 方法（可能只探索一条路径）
print("\n📌 对比：CoT 方法（线性思考）")
print("─"*80)
cot_prompt = f"""
用 {numbers_24} 这四个数字，通过加减乘除运算得到24。

让我们一步步思考：
"""
result_cot = get_completion(cot_prompt)
print(f"\n📊 输出长度: {len(result_cot)} 字符")

time.sleep(2)

# ToT 方法：多路径探索+评估+选择
print("\n📌 ToT 方法（树状探索）")
print("─"*80)
tot_prompt = f"""
用 {numbers_24} 这四个数字，通过加减乘除运算得到24。

请使用思维树（Tree of Thoughts）方法：

**步骤1：生成3个不同的思路**
为每个思路尝试不同的数字组合和运算顺序。

**步骤2：评估每个思路**
检查每个思路是否能得到24，并打分（0-10分）：
- 10分：正确得到24
- 5-9分：接近24但不完全正确
- 0-4分：结果差距较大

**步骤3：选择最优思路**
选择得分最高的思路作为最终答案。

请按以下格式输出：

---
思路1：[具体运算式]
计算过程：...
结果：...
评分：X/10
评估：[为什么给这个分数]

思路2：[具体运算式]
计算过程：...
结果：...
评分：X/10
评估：[为什么给这个分数]

思路3：[具体运算式]
计算过程：...
结果：...
评分：X/10
评估：[为什么给这个分数]

---
最优选择：[选择得分最高的思路]
最终答案：[完整运算式]
"""
result_tot = get_completion(tot_prompt)
print(f"\n📊 输出长度: {len(result_tot)} 字符")

print("\n" + "="*80)
print("💡 观察要点：")
print("  - CoT：可能只尝试一种组合，失败了就没有后路")
print("  - ToT：探索多个组合，评估每个的可行性，选择最优解")
print("  - ToT 更适合需要\"试错\"的搜索问题")
print("="*80 + "\n")

🔬 实验二：ToT（思维树）- 多路径探索
测试问题：用 4, 9, 10, 13 这四个数字通过加减乘除得到 24

📌 对比：CoT 方法（线性思考）
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 我们尝试用 4, 9, 10, 13 这四个数字，通过加减乘除运算得到 24。我们一步步分析。

---

### 第一步：列出所有可能的组合和运算

我们有四个数字：4, 9, 10, 13

目标是：**24**

我们可以尝试一些基本的组合，比如：

#### 尝试 1: 用乘法和加减组合

我们先尝试一些组合，看看是否能得到 24。

---

### 尝试 1：

**13 + 10 = 23**  
**23 + 1 = 24**  
但我们没有 1，不过我们可以用 9 ÷ 4 = 2.25，不太行。

---

### 尝试 2：

**10 - 4 = 6**  
**9 + 13 = 22**  
**22 + 6 = 28** 不对

---

### 尝试 3：

**13 - 9 = 4**  
**10 - 4 = 6**  
**6 × 4 = 24** ✅

我们来验证一下：

- **13 - 9 = 4**
- **10 - 4 = 6**
- **6 × 4 = 24**

但是我们用了两个 4，这是不允许的。

---

### 尝试 4：

**13 - 9 = 4**  
**10 - 4 = 6**  
**6 × 4 = 24**

但我们又用了两个 4。

---

### 尝试 5：

**13 + 10 = 23**  
**9 - 4 = 5**  
**23 + 5 = 28** 不对

---

### 尝试 6：

**13 + 9 = 22**  
**10 - 4 = 6**  
**22 + 6 = 28** 不对

---

### 尝试 7：

**13 × (10 - 9) = 13 × 1 = 13**  
**13 + 4 = 17** 不对

---

### 尝试 8：

**13 - 10 = 3**  
**9 ÷ 3 = 3**  
**3 × 4 = 

---

# 实验三：GoT（思维图）- 思维聚合

## 📖 任务说明

**任务**：多角度分析复杂问题

**测试问题**：
> 评估"在城市中心建设大型购物中心"这一计划的可行性

## 🎯 GoT 工作流程

```
[经济角度] ──┐
              ├──> [聚合分析] ──> [综合建议]
[环境角度] ──┤          ↑
              |          |
[社会角度] ──┘          |
      ↓                 |
  [中间结果] ───────────┘
```

## 💡 核心机制

1. **生成（Generate）**：从多个独立角度分析问题
2. **聚合（Aggregate）**：合并不同角度的结论
3. **精炼（Refine）**：改进和综合，给出最终建议

## 🔍 对比 CoT、ToT 和 GoT

| 特性 | CoT | ToT | GoT |
|-----|-----|-----|-----|
| **结构** | 线性链 | 树状 | 图状 |
| **路径** | 单一 | 多条 | 网状 |
| **聚合** | ❌ | ❌ | ✅ |
| **适用** | 简单推理 | 搜索问题 | 复杂综合 |

In [11]:
# 实验三：CoT vs ToT vs GoT 综合对比

# 统一课题：创业决策问题
# 这个问题足够复杂，能体现三种方法的差异：
# - CoT：线性分析，一步步推导
# - ToT：多个方案并行探索，评估选优
# - GoT：多维度分析后聚合，形成综合判断

business_question = """
小王有50万存款，正在考虑是否辞职创业开一家奶茶店。
已知信息：
- 小王目前月薪2万，工作稳定
- 奶茶店预计投资40万（装修20万+设备10万+首批原料和押金10万）
- 选址在大学城附近，月租金1.5万
- 预计需要雇佣2名员工，月人工成本1万
- 周边已有3家奶茶店，竞争激烈
- 大学城有2万学生，消费能力中等
- 奶茶平均售价15元，成本约5元
- 小王没有餐饮行业经验，但很有热情

问题：小王应该辞职创业吗？请给出分析和建议。
"""

print("="*80)
print("🔬 实验三：CoT vs ToT vs GoT 综合对比")
print("="*80)
print(f"统一课题：{business_question.strip()}")
print("="*80)

# ============================================================
# 方法1：CoT（思维链）- 线性推理
# ============================================================
print("\n" + "🔗"*40)
print("📌 方法1：CoT（思维链）- 线性逐步推理")
print("🔗"*40)

cot_prompt = f"""
{business_question}

请使用思维链方法，一步步分析这个问题：

第一步：分析财务可行性
（计算投资回报、盈亏平衡点等）

第二步：分析市场风险
（考虑竞争、客源等因素）

第三步：分析个人因素
（经验、机会成本等）

第四步：综合判断
（基于以上分析得出结论）

最终建议：
"""

print("─"*80)
result_cot = get_completion(cot_prompt)
print(f"\n📊 输出长度: {len(result_cot)} 字符")

time.sleep(3)

# ============================================================
# 方法2：ToT（思维树）- 多方案探索与评估
# ============================================================
print("\n" + "🌳"*40)
print("📌 方法2：ToT（思维树）- 多方案并行探索")
print("🌳"*40)

tot_prompt = f"""
{business_question}

请使用思维树方法，探索多个可能的决策方案：

**第一步：生成3个不同的决策方案**

方案A：[直接辞职全力创业]
具体计划：...
预期结果：...

方案B：[保持工作，兼职筹备]
具体计划：...
预期结果：...

方案C：[暂不创业，先积累经验]
具体计划：...
预期结果：...

**第二步：评估每个方案（0-10分）**

方案A评分：X/10
- 财务风险：...
- 成功概率：...
- 机会成本：...

方案B评分：X/10
- 财务风险：...
- 成功概率：...
- 机会成本：...

方案C评分：X/10
- 财务风险：...
- 成功概率：...
- 机会成本：...

**第三步：选择最优方案**

最优选择：方案X
选择理由：...

最终建议：
"""

print("─"*80)
result_tot = get_completion(tot_prompt)
print(f"\n📊 输出长度: {len(result_tot)} 字符")

time.sleep(3)

# ============================================================
# 方法3：GoT（思维图）- 多维度分析与聚合
# ============================================================
print("\n" + "🕸️"*40)
print("📌 方法3：GoT（思维图）- 多维度聚合分析")
print("🕸️"*40)

got_prompt = f"""
{business_question}

请使用思维图方法，从多个独立维度分析，然后聚合形成结论：

**第一步：多维度独立分析**

<财务维度>
投资回报分析：
- 总投资额：...
- 月固定成本：...
- 单杯利润：...
- 盈亏平衡点（日销量）：...
- 回本周期预估：...
财务结论：[有利/不利/中性]，理由：...
</财务维度>

<市场维度>
市场竞争分析：
- 目标客群规模：...
- 竞争对手数量和特点：...
- 差异化空间：...
- 市场饱和度判断：...
市场结论：[有利/不利/中性]，理由：...
</市场维度>

<个人维度>
个人条件分析：
- 行业经验：...
- 资金储备vs投资需求：...
- 机会成本（放弃的稳定收入）：...
- 风险承受能力：...
个人结论：[有利/不利/中性]，理由：...
</个人维度>

**第二步：维度间关联分析**

<关联分析>
- 财务维度与市场维度的交叉影响：...
- 个人维度对成功率的影响：...
- 三个维度中最关键的制约因素：...
</关联分析>

**第三步：聚合与综合决策**

<聚合结论>
综合评估：
- 有利因素汇总：...
- 不利因素汇总：...
- 核心矛盾：...

最终建议：[建议创业 / 不建议创业 / 有条件创业]
具体行动方案：...
</聚合结论>
"""

print("─"*80)
result_got = get_completion(got_prompt)
print(f"\n📊 输出长度: {len(result_got)} 字符")

# ============================================================
# 实验总结
# ============================================================
print("\n" + "="*80)
print("📊 三种方法对比总结")
print("="*80)
print(f"""
┌─────────┬──────────────────────────────────────────────────┐
│  方法   │  特点                                            │
├─────────┼──────────────────────────────────────────────────┤
│  CoT    │  线性推理，按步骤逐层深入                        │
│         │  适合：因果链清晰的问题                          │
├─────────┼──────────────────────────────────────────────────┤
│  ToT    │  并行探索多个方案，评估后选优                    │
│         │  适合：需要比较多个选项的决策问题                │
├─────────┼──────────────────────────────────────────────────┤
│  GoT    │  多维度独立分析，交叉关联后聚合                  │
│         │  适合：多因素复杂问题，需综合权衡                │
└─────────┴──────────────────────────────────────────────────┘

📝 观察要点：
1. CoT 产出一个线性推理链，结论由前序步骤决定
2. ToT 产出多个并行方案，通过评分机制筛选
3. GoT 产出多个维度分析，通过聚合形成综合判断

💡 同一问题，三种方法可能得出相同或不同的结论，
   但推理路径和分析深度有本质区别。
""")

🔬 实验三：CoT vs ToT vs GoT 综合对比
统一课题：小王有50万存款，正在考虑是否辞职创业开一家奶茶店。
已知信息：
- 小王目前月薪2万，工作稳定
- 奶茶店预计投资40万（装修20万+设备10万+首批原料和押金10万）
- 选址在大学城附近，月租金1.5万
- 预计需要雇佣2名员工，月人工成本1万
- 周边已有3家奶茶店，竞争激烈
- 大学城有2万学生，消费能力中等
- 奶茶平均售价15元，成本约5元
- 小王没有餐饮行业经验，但很有热情

问题：小王应该辞职创业吗？请给出分析和建议。

🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗
📌 方法1：CoT（思维链）- 线性逐步推理
🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗🔗
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 我们采用**思维链（Chain of Thought）**的方法，逐步分析小王是否应该辞职创业开奶茶店。我们将从**财务可行性**、**市场风险**、**个人因素**三个方面进行分析，最后综合判断。

---

## **第一步：分析财务可行性**

### 1. **初始投资分析**
- **总投资**：40万元（装修20万 + 设备10万 + 首批原料和押金10万）
- **小王现有存款**：50万元 → 足够支付初始投资

### 2. **每月固定成本**
- **租金**：1.5万元
- **人工成本**：1万元（2名员工）
- **其他成本**（如水电、杂费等）：估计为1万元
- **总固定成本**：1.5 + 1 + 1 = **3.5万元/月**

### 3. **每杯奶茶的利润**
- **售价**：15元
- **成本**：5元
- **单杯利润**：15 - 5 = **10元**

### 4. **日均销量估算**
- **大学城有2万学生**，消费能力中等
- 假设每人每月消费10杯 → 2万 × 10 = **20万杯/月**
- 但实际不可能全部消费，假设实际销量为 **30%** → 20万 × 30% = **6万杯/月**

### 5. *

---

# 🎓 总结与最佳实践

## 📊 三种方法完整对比

| 维度 | CoT | ToT | GoT |
|-----|-----|-----|-----|
| **提出时间** | 2022 | 2023 | 2023 |
| **结构** | 线性链条 | 树状结构 | 图状网络 |
| **推理路径** | 单一路径 | 多路径+剪枝 | 网状+聚合 |
| **是否回溯** | ❌ 不支持 | ✅ 支持 | ✅ 支持 |
| **是否聚合** | ❌ 不支持 | ❌ 不支持 | ✅ 支持 |
| **模型调用** | 1次 | 1次（模拟多路径） | 1-3次 |
| **实现复杂度** | ⭐ 简单 | ⭐⭐ 中等 | ⭐⭐⭐ 复杂 |
| **token 成本** | 低 | 中 | 高 |
| **适用场景** | 线性推理 | 搜索问题 | 复杂综合 |

## 🎯 Qwen3 思考模式控制（重要补充）

Qwen3 是一个特殊的推理模型，**支持开启/关闭思考模式**，让我们能够清晰对比提示词工程和内置推理能力的差异。

### 控制方法

#### 1. **Python API**（推荐）
```python
from ollama import chat

# 关闭思考（作为普通模型）
chat(model='qwen3', messages=[...], think=False)

# 开启内置思考（推理模式）
chat(model='qwen3', messages=[...], think=True)
```

#### 2. **命令行方式**
```bash
# 启用思考
ollama run qwen3 --think "问题"

# 禁用思考
ollama run qwen3 --think=false "问题"
```

#### 3. **交互式会话**
```bash
# 进入交互模式后
/set think      # 启用思考
/set nothink    # 禁用思考（或 /no_think）
```

### 三种模式对比

| 模式 | 配置 | 推理方式 | 适用场景 |
|-----|------|---------|---------|
| **普通模式** | `think=False` | 直接回答 | 简单问题 |
| **提示词 CoT** | `think=False` + "Let's think step by step" | 用户引导推理 | 中等复杂度 |
| **内置思考** | `think=True` | 模型自主推理+验证 | 复杂推理任务 |

### 关键差异

**提示词 CoT**（`think=False` + prompt）：
- 表面上的逐步输出
- ❌ 无自我验证
- ❌ 无真实回溯
- ✅ 适用于所有 LLM

**内置思考**（`think=True`）：
- 模型内部深度思考
- ✅ 自动验证答案
- ✅ 可以回溯重试
- ⚠️ 只有推理模型支持

### 成本对比

```python
# 成本从低到高：
1. think=False（普通模式）         成本: 1x
2. think=False + CoT prompt（提示词） 成本: ~2x
3. think=True（内置思考）          成本: ~3-5x（思考token计费）
```

## 💡 使用决策树

```
                     [需要推理吗？]
                          |
          +---------------+---------------+
          |                               |
        需要                          不需要
          |                               ↓
    [问题类型？]                    直接回答即可
          |
    +-----+-----+-----+
    |     |     |     |
  线性  搜索  综合  创意
    ↓     ↓     ↓     ↓
   CoT   ToT   GoT   GoT
    |
    +--- 如果有推理模型 ---+
    |                      |
提示词 CoT          内置思考(think=True)
 (便宜)              (更可靠但贵)
```

### 1. **使用 CoT** - 线性推理问题

**适用场景**：
- ✅ 数学计算、逻辑推理
- ✅ 步骤明确的操作指南
- ✅ 简单的因果分析

**实现方式**：
- **Zero-shot CoT**: 添加 "Let's think step by step"
- **Few-shot CoT**: 提供1-2个带推理过程的示例

**示例**：
```python
prompt = """
问题：[复杂数学题]

让我们一步步思考：
"""
```

### 2. **使用 ToT** - 搜索和优化问题

**适用场景**：
- ✅ 需要试错的问题（24点、数独）
- ✅ 多种可能方案的选择（路径规划）
- ✅ 创意生成（需要评估多个创意）

**实现方式**：
- 在提示词中要求模型生成多个思路
- 让模型自己评估每个思路
- 选择得分最高的思路

**示例**：
```python
prompt = """
问题：[需要试错的问题]

请生成3个不同的思路：
思路1：... 评分：X/10
思路2：... 评分：X/10
思路3：... 评分：X/10

选择得分最高的作为答案。
"""
```

### 3. **使用 GoT** - 复杂综合问题

**适用场景**：
- ✅ 需要多角度分析（政策评估、战略决策）
- ✅ 需要聚合多个信息源（文档总结、比较分析）
- ✅ 迭代优化（文章写作、代码重构）

**实现方式**：
- 将问题分解为多个独立子问题
- 分别分析每个子问题
- 聚合所有结论给出最终答案

**示例**：
```python
prompt = """
从以下角度分析问题：
<angle1>...</angle1>
<angle2>...</angle2>
<angle3>...</angle3>

聚合分析：
<aggregation>综合以上角度...</aggregation>

最终建议：...
"""
```

## 🆚 与推理模型（o1/R1/Qwen3）的关键区别

| 维度 | CoT/ToT/GoT（提示词） | Qwen3（think=True） | o1/DeepSeek-R1 |
|------|----------------------|---------------------|----------------|
| **本质** | 用户引导模型 | 模型内置能力（可控） | 模型内置能力（Always-On） |
| **实现** | 提示工程技巧 | 训练时植入+开关控制 | 训练时植入 |
| **验证** | ❌ 无自我验证 | ✅ 内置验证 | ✅ 内置验证 |
| **回溯** | ❌ 需多轮模拟 | ✅ 真实回溯 | ✅ 真实回溯 |
| **思考控制** | ✅ 完全可控 | ✅ 可开关（think参数） | ❌ 无法关闭 |
| **透明度** | ✅ 推理过程可见 | ✅ <think>标签可见 | ⚠️ 部分隐藏（o1）|
| **成本** | 中等（prompt长度） | 高（思考token计费） | 高（思考token计费） |
| **适用模型** | 所有 LLM | Qwen3系列 | o1/R1系列 |

**形象比喻**：
- **CoT/ToT/GoT**：给普通人一张详细的解题步骤清单
- **Qwen3 (think=True)**：训练出会独立思考的数学天才，并且可以选择让他展示思考过程或直接给答案
- **o1/R1**：永远在深度思考的数学天才（无法关闭思考）

## ⚠️ 常见误区与解决方案

| 误区 | 问题 | 正确做法 |
|-----|------|---------|
| 所有问题都用 CoT | 简单问题浪费 token | 简单问题直接问 |
| ToT 总是比 CoT 好 | 线性问题不需要树状探索 | 根据问题类型选择 |
| GoT 需要复杂框架 | 实现门槛高 | 简化版：多角度分析+聚合 |
| 以为能替代推理模型 | 提示词无法实现真正回溯 | 理解两者的本质区别 |
| Qwen3 一直开 think | 简单问题成本高 | 根据任务复杂度开关 |

## 🚀 实战技巧

### 1. **渐进式增强**

```python
# 第1步：先试直接回答（Qwen3 think=False）
result = model("问题", think=False)

# 第2步：效果不好，加 CoT 提示词
result = model("问题\n\nLet's think step by step.", think=False)

# 第3步：还不够，开启 Qwen3 内置思考
result = model("问题", think=True)

# 第4步：极其复杂，升级到 ToT/GoT
result = model("问题\n\n生成3个思路，评估后选择最优...", think=True)
```

### 2. **混合使用**

```python
# CoT + ToT：先用 CoT 生成思路，再用 ToT 评估
prompt = """
[Few-shot CoT 示例]

现在生成3个思路并评估：...
"""

# Qwen3 think + GoT：内置思考 + 多角度聚合
prompt = """
从角度1、2、3分析（开启内置思考）：...
聚合最优思路：...
"""
chat(model='qwen3', messages=[...], think=True)
```

### 3. **成本优化**

| 方法 | Token 成本 | 优化建议 |
|-----|-----------|---------|
| Zero-shot | ~1x 基础 | 简单问题首选 |
| CoT 提示词 | ~2x 基础 | Zero-shot 优先 |
| ToT 提示词 | ~3-5x 基础 | 限制思路数量（3个足够） |
| GoT 提示词 | ~5-10x 基础 | 只在真正需要聚合时使用 |
| **Qwen3 think=True** | ~3-8x 基础 | 复杂推理任务使用 |
| **o1/R1（Always-On）** | ~10-20x 基础 | 最复杂的任务 |

## 📈 实际应用场景

### 场景1：客服机器人（CoT）

```python
# 问题：用户投诉订单未到货
prompt = """
用户问题：订单10天了还没到货

请一步步处理：
1. 查询订单状态
2. 判断是否超时
3. 给出解决方案
"""
```

### 场景2：代码调试（ToT + Qwen3 think）

```python
# 问题：代码报错，尝试多种修复方案
prompt = """
错误：[报错信息]

生成3个修复方案：
方案1：... 可行性：X/10
方案2：... 可行性：X/10
方案3：... 可行性：X/10

选择最可行的方案。
"""
# 开启 Qwen3 内置思考增强推理
chat(model='qwen3', messages=[...], think=True)
```

### 场景3：商业决策（GoT）

```python
# 问题：是否进入新市场
prompt = """
从以下角度分析：
<market>市场角度：市场规模、竞争态势</market>
<finance>财务角度：投资成本、预期收益</finance>
<risk>风险角度：政策风险、运营风险</risk>

聚合分析并给出决策建议。
"""
```

## 🔮 未来展望

### 1. **推理模型的演进（2024-2025）**

- ✅ **o1、DeepSeek-R1**：内置 CoT，Always-On，最强但最贵
- ✅ **Qwen3**：可控思考模式（think 参数），平衡性能和成本
- 🔮 **未来趋势**：更多模型支持可控思考，降低推理成本

### 2. **提示词工程的价值依然存在**

即使有推理模型，提示词 CoT/ToT/GoT 仍然有价值：
- 💰 **成本优势**：在普通模型上实现类似效果
- 🎯 **可控性**：明确指定推理结构
- 🔍 **透明性**：推理过程完全可见
- 🔧 **灵活性**：可与推理模型的内置能力结合

### 3. **最佳实践（2025+）**

```python
# 简单任务：直接用普通模型
result = claude("简单问题")

# 中等任务：提示词 CoT
result = claude("问题\n\nLet's think step by step.")

# 复杂任务（成本敏感）：Qwen3 可控思考
result = chat(model='qwen3', messages=[...], think=True)

# 极复杂任务（追求最佳）：o1/R1
result = deepseek_r1("复杂问题")  # 模型自己会深度思考
```

---

## 🎉 实验完成

通过四个实验（含额外实验），你已经掌握了：

✅ **CoT**：如何用 "Let's think step by step" 引导线性推理  
✅ **ToT**：如何让模型探索多条路径并选择最优解  
✅ **GoT**：如何从多角度分析并聚合结论  
✅ **Qwen3 思考模式**：如何控制内置思考的开关（think 参数）  
✅ 提示词 CoT vs 内置思考的本质区别  
✅ 三种方法的区别和适用场景  

**关键要点**：
- **CoT** = 一条路走到底（适合线性推理）
- **ToT** = 多条路试试看（适合搜索问题）
- **GoT** = 多个角度综合（适合复杂分析）
- **提示词技巧** = 给普通人一张解题清单
- **推理模型（可控）** = Qwen3 think 参数
- **推理模型（Always-On）** = o1/R1 永远在思考

💡 **下一步建议**：
- 在实际项目中根据任务类型选择合适方法
- 尝试混合使用（CoT + ToT、提示词 + think=True）
- 根据成本和性能需求选择合适的模型和模式
- 关注推理模型的发展，适时升级技术栈

## 📚 参考资源

- [Ollama 思考模式文档](https://docs.ollama.com/capabilities/thinking)
- [Qwen3 模型页面](https://ollama.com/library/qwen3)
- CoT 论文：Chain-of-Thought Prompting (2022)
- ToT 论文：Tree of Thoughts (2023)
- GoT 论文：Graph of Thoughts (2023)